<a href="https://colab.research.google.com/github/IraStreltsova/MissionSquare_Surveys/blob/main/SURVEY_HISTORICAL_DATA_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
# Constants
sat_columns = ['TOP 2 BOX %', 'AVG SCORE', 'n']
nps_columns = ['NPS', 'n']
EXCEL_RESULTS = 'SURVEY_HISTORICAL_RESULTS.xlsx'

In [ ]:
# Survey Data Configuration
survey_config = {
    'GPAS ADVISOR': {
        'file_path': 'GPAS Adviser Survey Results July 2023.xlsx',
        'question_column': ['Q2'],
        'distribution': {
            'Exceeds my expectations': 3,
            'Meets my expectations': 2,
            'Does not meet my expectations': 1
        },
        'valid_scores': [1, 2, 3]
    },
    'CFP CONSULTATIONS': {
        'file_path': 'CFP Consultations Results July 2023.xlsx',
        'question_column': ['Q31'],
        'distribution': {
            'Exceeded my expectations': 3,
            'Met my expectations': 2,
            'Did not meet my expectations': 1
        },
        'valid_scores': [1, 2, 3]
    },
    'FINANCIAL PLAN DELIVERY RECOMMEND': {
        'file_path': 'Financial Plan Survey Results July 2023.xlsx',
        'question_column': ['Q2_6'],
        'distribution': {
            'Strongly agree' : 5,
            'Agree' : 4,
            'Neither agree nor disagree': 3,
            'Disagree': 2,
            'Strongly disagree': 1
        },
        'valid_scores': [1, 2, 3, 4, 5]
    },
    'FINANCIAL PLAN DELIVERY OVERALL': {
        'file_path': 'Financial Plan Survey Results July 2023.xlsx',
        'question_column': ['Q6'],
        'distribution': {
            'Overly exceeds my expectations' : 4,
            'Exceeds my expectations': 3,
            'Meets my expectations': 2,
            'Does not meet my expectations': 1
        },
        'valid_scores': [1, 2, 3, 4]
    },
    'VRS ACCOUNT REVIEW REP': {
        'file_path': 'VRS AR Survey Results July 2023.xlsx',
        'question_column': ['Q18_5'],
        'distribution': {
            'Strongly Agree' : 4,
            'Agree' : 3,
            'Disagree': 2,
            'Strongly disagree': 1
        },
        'valid_scores': [1, 2, 3, 4]
    },
    'VRS ACCOUNT REVIEW EXP': {
        'file_path': 'VRS AR Survey Results July 2023.xlsx',
        'question_column': ['Q19_4'],
        'distribution': {
            'Strongly Agree' : 4,
            'Agree' : 3,
            'Disagree': 2,
            'Strongly disagree': 1
        },
        'valid_scores': [1, 2, 3, 4]
    },
    'VRS SEMINAR/WEBINAR PRESENTER': {
        'file_path': 'VRS Seminar Webinar Survey Results July 2023.xlsx',
        'question_column': ['Q14_4'],
        'distribution': {
            'Strongly Agree' : 4,
            'Agree' : 3,
            'Disagree': 2,
            'Strongly disagree': 1
        },
        'valid_scores': [1, 2, 3, 4]
    },
    'VRS SEMINAR/WEBINAR PRESENTATION': {
        'file_path': 'VRS Seminar Webinar Survey Results July 2023.xlsx',
        'question_column': ['Q15_4'],
        'distribution': {
            'Strongly Agree' : 4,
            'Agree' : 3,
            'Disagree': 2,
            'Strongly disagree': 1
        },
        'valid_scores': [1, 2, 3, 4]
    },
}

In [ ]:
# NPS VALUES
nps_config = {
    'FINANCIAL PLAN DELIVERY NPS': {
        'file_path': 'Financial Plan Survey Results July 2023.xlsx',
        'question_column': ['Q7_NPS_GROUP'],
        'distribution': {
            'Promoter': 3,
            'Passive': 2,
            'Detractor': 1
        },
        'valid_scores': [1, 2, 3]
    },
}

In [ ]:
def read_and_clean_data(file_path, column_to_keep):
  """Read and clean data from the Excel file."""
  df = pd.read_excel(file_path)
  df = df.drop(0).reset_index(drop=True)
  df = df[column_to_keep]
  return df

In [ ]:
def process_survey(file_name, question_column, distribution, valid_scores):

    df = read_and_clean_data(file_name, question_column)

    # Create an empty DataFrame
    result_df = pd.DataFrame(columns=sat_columns)

    # Add 'DISTRIBUTION' column to the DataFrame
    df['DISTRIBUTION'] = df[question_column].replace(distribution)

    # Filter rows where DISTRIBUTION is in valid scores
    filtered_data = df[df['DISTRIBUTION'].isin(valid_scores)]

    # Calculate average score
    avg_score = filtered_data['DISTRIBUTION'].mean()

    # Group column 'DISTRIBUTION' and calculate percentages
    distribution = filtered_data['DISTRIBUTION'].value_counts(normalize=True)

    # Calculate percentages for 'TOP 1 BOX %' and 'TOP 2 BOX %'
    top_1_box = distribution.get(valid_scores[-1], 0)
    top_2_box = top_1_box + distribution.get(valid_scores[-2], 0)

    # Count rows for 'n'
    n_count = len(filtered_data)

    # Append data to the result DataFrame
    result_df = result_df.append({
        'TOP 2 BOX %': top_2_box,
        'AVG SCORE': avg_score,
        'n': n_count
    }, ignore_index=True)

    return result_df

In [ ]:
# FINANCIAL PLAN DELIVERY (NPS)
def calculate_nps(file_name, question_column, distribution, valid_scores):
  # Read the Excel file into a DataFrame
  df = read_and_clean_data(file_name, question_column)

  nps_df = pd.DataFrame(columns=nps_columns)

  df['DISTRIBUTION'] = df[question_column].replace(distribution)

  # Filter rows where DISTRIBUTION is in valid scores
  filtered_data = df[df['DISTRIBUTION'].isin(valid_scores)]


  # NPS
  # Group column '' and calculate percentages
  distribution = filtered_data['DISTRIBUTION'].value_counts(normalize=True)*100

  # Calculate percentages for NPS
  promoter = distribution.get(valid_scores[-1], 0)
  detractor = distribution.get(valid_scores[0], 0)
  nps = promoter - detractor


  # NUMBER OF ROWS
  # Count rows for 'n'
  n_count = len(filtered_data)


  # APPEND DATA to the 'fpd_rec' DataFrame
  nps_df = nps_df.append({
      'NPS': nps,
      'n': n_count
  }, ignore_index=True)

  return nps_df

In [ ]:
# EXPORTING DATA TO EXCEL
def export_to_excel(merged, nps, file_path):
  with pd.ExcelWriter(file_path) as writer:
    # Export DataFrames to different sheets in the Excel file
    merged.to_excel(writer, sheet_name='merged_dataframe', index=True)
    nps.to_excel(writer, sheet_name='NPS', index=True)

In [ ]:
def main():
    dataframes = {}
    for survey_name, config in survey_config.items():
        df = process_survey(config['file_path'], config['question_column'], config['distribution'], config['valid_scores'])
        dataframes[survey_name] = df

    merged_dataframe = pd.concat(dataframes.values(), keys=survey_config.keys())
    merged_dataframe = merged_dataframe.reset_index(level=1, drop=True)

    nps_df = {}
    for survey_name, config in nps_config.items():
        df = calculate_nps(config['file_path'], config['question_column'], config['distribution'], config['valid_scores'])
        nps_df[survey_name] = df

    fpd_nps = pd.concat(nps_df.values(), keys=nps_config.keys())
    fpd_nps = fpd_nps.reset_index(level=1, drop=True)

    export_to_excel(merged_dataframe, fpd_nps, EXCEL_RESULTS)

In [ ]:
if __name__ == "__main__":
    main()

<ipython-input-98-36f435b8fc83>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
<ipython-input-98-36f435b8fc83>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
<ipython-input-98-36f435b8fc83>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
<ipython-input-98-36f435b8fc83>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({
<ipython-input-98-36f435b8fc83>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df

In [ ]:
# # Constants
# gpas_question = ['Q2']
# cfp_question = ['Q31']
# fpd_rec_question = ['Q2_6']
# fpd_exp_question = ['Q6']
# vrs_rep_question = ['Q18_5']
# vrs_exp_question = ['Q19_4']
# vrs_sw_presr_question = ['Q14_4']
# vrs_sw_presn_question = ['Q15_4']
# nps_question = ['Q7_NPS_GROUP']
# VALID_NPS_SCORES = [1, 2, 3]
# VALID_GPAS_SCORES = [1, 2, 3]
# VALID_CFP_SCORES = [1, 2, 3]
# VALID_FPD_REC_SCORES = [1, 2, 3, 4, 5]
# VALID_FPD_EXP_SCORES = [1, 2, 3, 4]
# VALID_VRS_SCORES = [1, 2, 3, 4]
# sat_columns = ['TOP 2 BOX %', 'AVG SCORE', 'n']
# nps_columns = ['NPS', 'n']
# EXCEL_RESULTS = 'SURVEY_HISTORICAL_RESULTS.xlsx'

In [ ]:
# gpas_distribution = {
#     'Exceeds my expectations': 3,
#     'Meets my expectations': 2,
#     'Does not meet my expectations': 1
# }

# cfp_distribution = {
#     'Exceeded my expectations': 3,
#     'Met my expectations': 2,
#     'Did not meet my expectations': 1
# }

# fpd_rec_distribution = {
#     'Strongly agree' : 5,
#     'Agree' : 4,
#     'Neither agree nor disagree': 3,
#     'Disagree': 2,
#     'Strongly disagree': 1
# }

# fpd_exp_distribution = {
#     'Overly exceeds my expectations' : 4,
#     'Exceeds my expectations': 3,
#     'Meets my expectations': 2,
#     'Does not meet my expectations': 1
# }

# nps_distribution = {
#     'Promoter': 3,
#     'Passive': 2,
#     'Detractor': 1
# }

# vrs_distribution = {
#     'Strongly Agree' : 4,
#     'Agree' : 3,
#     'Disagree': 2,
#     'Strongly disagree': 1
# }

In [ ]:
# def merge_dataframes(dataframes):
#     # Create an empty list to hold the DataFrame objects
#     dataframes_list = []

#     # Iterate through the dictionary and add index names as a level of MultiIndex
#     for index_name, dataframe in dataframes.items():
#         dataframe.index = pd.MultiIndex.from_product([[index_name], dataframe.index])
#         dataframes_list.append(dataframe)

#     # Concatenate the list of DataFrames into one DataFrame
#     merged_dataframe = pd.concat(dataframes_list)

#     merged_dataframe = merged_dataframe.droplevel(level=1, axis=0)
#     return merged_dataframe

In [ ]:
# def main():

#   gpas = process_survey('GPAS Adviser Survey Results July 2023.xlsx', gpas_question, gpas_distribution, VALID_GPAS_SCORES)
#   cfp = process_survey('CFP Consultations Results July 2023.xlsx', cfp_question, cfp_distribution, VALID_CFP_SCORES)
#   fpd_rec = process_survey('Financial Plan Survey Results July 2023.xlsx', fpd_rec_question, fpd_rec_distribution, VALID_FPD_REC_SCORES)
#   fpd_exp = process_survey('Financial Plan Survey Results July 2023.xlsx', fpd_exp_question, fpd_exp_distribution, VALID_FPD_EXP_SCORES)
#   fpd_nps = calculate_nps('Financial Plan Survey Results July 2023.xlsx', nps_question, nps_distribution, VALID_NPS_SCORES)
#   vrs_rep = process_survey('VRS AR Survey Results July 2023.xlsx', vrs_rep_question, vrs_distribution, VALID_VRS_SCORES)
#   vrs_exp = process_survey('VRS AR Survey Results July 2023.xlsx', vrs_exp_question, vrs_distribution, VALID_VRS_SCORES)
#   vrs_sw_presr = process_survey('VRS Seminar Webinar Survey Results July 2023.xlsx', vrs_sw_presr_question, vrs_distribution, VALID_VRS_SCORES)
#   vrs_sw_presn = process_survey('VRS Seminar Webinar Survey Results July 2023.xlsx', vrs_sw_presn_question, vrs_distribution, VALID_VRS_SCORES)

#   # Create a dictionary with DataFrame objects and their respective names
#   dataframes = {
#     'GPAS ADVISOR': gpas,
#     'CFP CONSULTATIONS': cfp,
#     'FINANCIAL PLAN DELIVERY RECOMMEND': fpd_rec,
#     'FINANCIAL PLAN DELIVERY OVERALL': fpd_exp,
#     'VRS ACCOUNT REVIEW REP':vrs_rep,
#     'VRS ACCOUNT REVIEW EXP':vrs_exp,
#     'VRS SEMINAR/WEBINAR PRESENTER':vrs_sw_presr,
#     'VRS SEMINAR/WEBINAR PRESENTATION':vrs_sw_presn
#   }

#   merged_dataframe = merge_dataframes(dataframes)
#   export_to_excel(merged_dataframe, fpd_nps, EXCEL_RESULTS)